In [2]:
# In this approach user can find the forecast for all the parts which will covers maximum variations
# Create feature using dates
# Apply TSFRESH framework to create multiple features
# Preporcessing for the machine learning algorithm
# Apply different algorithms
# Chose the algorithm of best accuracy

In [ ]:
# -*- coding: utf-8 -*-
"""
Use Case 2: Build a Time Series Forecast Model for Inventory Management

author: RK
"""
import warnings
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
df = pd.read_excel('Usecase2_Dataset.xlsx', header=[0,1,2])

df.columns = df.columns.map('{0[0]}{0[1]}{0[2]}'.format)

df=df.groupby(['Unnamed: 0_level_0Unnamed: 0_level_1Part No']).sum() # Aggeregate redundant Parts
#df.head()


# take transpose, easy to work with
df = df.T

# use datetime for index
xx = pd.date_range(start='1/1/2016', periods=177, freq='W')
df = df.set_index(xx)

df = df.apply(pd.to_numeric, errors='coerce')
df = df.fillna(0)

df.head()


In [4]:
df1=df.reset_index()
df1.columns.values[0]='date'
#df1=df1[['date',29032395]]
df1.head()

Unnamed: 0_level_0Unnamed: 0_level_1Part No,date,18345617,18345618,19316005,19316006,29032395,29032396,29032407,29032408,29032411,...,32054388EG,32054389EG,A005H177,A005H178,A005Y137,A005Y138,A008A576,A008A576EG,A008A578,A008A578EG
0,2016-01-03,0,0,0,0,24,24,200,200,0,...,0,0,0,0,0,0,0,0,0,0
1,2016-01-10,0,0,0,0,214,210,692,692,0,...,0,0,0,0,0,0,0,0,0,0
2,2016-01-17,0,0,0,0,162,162,864,868,0,...,0,0,0,0,0,0,0,0,0,0
3,2016-01-24,0,0,0,0,30,34,764,765,100,...,0,0,0,0,0,0,0,0,0,0
4,2016-01-31,0,0,0,0,60,60,784,784,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df = pd.melt(df1, id_vars =['date'])
df.columns=['Date','Category','Target']
df.head()

from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
df["ID"] = lb_make.fit_transform(df["Category"].astype(str))

df=df.groupby(['ID','Date','Category']).sum().reset_index()
df['Date'] =pd.to_datetime(df.Date)
df = df.sort_values(by=['ID','Date']).reset_index()
df = df[['ID','Date','Target']]
#df1=df.sort_values(by='Date')
df

,ID,Date,Target
0,0,2016-01-03,0
1,0,2016-01-10,0
2,0,2016-01-17,0
3,0,2016-01-24,0
4,0,2016-01-31,0
...,...,...,...
33625,189,2019-04-21,0
33626,189,2019-04-28,396
33627,189,2019-05-05,396
33628,189,2019-05-12,264


In [6]:
import datetime as dt
df['Year'] = df.Date.dt.year
df["Quarter"] = df.Date.dt.quarter
df["Month"] = df.Date.dt.month

In [7]:
df.head()

,ID,Date,Target,Year,Quarter,Month
0,0,2016-01-03,0,2016,1,1
1,0,2016-01-10,0,2016,1,1
2,0,2016-01-17,0,2016,1,1
3,0,2016-01-24,0,2016,1,1
4,0,2016-01-31,0,2016,1,1


In [8]:
# create feature using featuretools
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import roll_time_series, make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute


df_rolled = roll_time_series(df, column_id="ID", column_sort="Date")

Rolling: 100%|██████████| 20/20 [01:42<00:00,  5.13s/it]


In [9]:
# Feature with customized setting
settings = {
    "abs_energy": None,
    "agg_autocorrelation": [
        {"f_agg": "mean", "maxlag": 52}, 
        {"f_agg": "var", "maxlag": 52}
    ]
}


df_features = extract_features(df_rolled, column_id="id", column_sort="Date",default_fc_parameters=settings)

Feature Extraction: 100%|██████████| 20/20 [00:55<00:00,  2.77s/it]


In [10]:
x=df_features.reset_index().sort_index()
x['Date']=df['Date']
x['Target']=df['Target']
x['ID'] =df['ID']
x

variable,id,ID__abs_energy,"ID__agg_autocorrelation__f_agg_""mean""__maxlag_52","ID__agg_autocorrelation__f_agg_""var""__maxlag_52",Month__abs_energy,"Month__agg_autocorrelation__f_agg_""mean""__maxlag_52","Month__agg_autocorrelation__f_agg_""var""__maxlag_52",Quarter__abs_energy,"Quarter__agg_autocorrelation__f_agg_""mean""__maxlag_52","Quarter__agg_autocorrelation__f_agg_""var""__maxlag_52",Target__abs_energy,"Target__agg_autocorrelation__f_agg_""mean""__maxlag_52","Target__agg_autocorrelation__f_agg_""var""__maxlag_52",Year__abs_energy,"Year__agg_autocorrelation__f_agg_""mean""__maxlag_52","Year__agg_autocorrelation__f_agg_""var""__maxlag_52",Date,Target,ID
0,"id=0,timeshift=2016-01-03 00:00:00",0.0,0.0,0.0,1.0,0.000000,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.000000,4064256.0,0.000000,0.000000,2016-01-03,0,0
1,"id=0,timeshift=2016-01-10 00:00:00",0.0,0.0,0.0,2.0,0.000000,0.000000,2.0,0.000000,0.000000,0.0,0.000000,0.000000,8128512.0,0.000000,0.000000,2016-01-10,0,0
2,"id=0,timeshift=2016-01-17 00:00:00",0.0,0.0,0.0,3.0,0.000000,0.000000,3.0,0.000000,0.000000,0.0,0.000000,0.000000,12192768.0,0.000000,0.000000,2016-01-17,0,0
3,"id=0,timeshift=2016-01-24 00:00:00",0.0,0.0,0.0,4.0,0.000000,0.000000,4.0,0.000000,0.000000,0.0,0.000000,0.000000,16257024.0,0.000000,0.000000,2016-01-24,0,0
4,"id=0,timeshift=2016-01-31 00:00:00",0.0,0.0,0.0,5.0,0.000000,0.000000,5.0,0.000000,0.000000,0.0,0.000000,0.000000,20321280.0,0.000000,0.000000,2016-01-31,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33625,"id=99,timeshift=2019-04-21 00:00:00",1695573.0,0.0,0.0,8688.0,-0.010759,0.193807,1219.0,-0.009573,0.217809,19854765.0,0.305404,0.028314,703943253.0,0.586222,0.041860,2019-04-21,0,189
33626,"id=99,timeshift=2019-04-28 00:00:00",1705374.0,0.0,0.0,8704.0,-0.011069,0.194605,1223.0,-0.009716,0.218257,20014765.0,0.306603,0.028079,708019614.0,0.588500,0.041665,2019-04-28,396,189
33627,"id=99,timeshift=2019-05-05 00:00:00",1715175.0,0.0,0.0,8729.0,-0.011252,0.194942,1227.0,-0.009865,0.218618,20490865.0,0.306409,0.026909,712095975.0,0.590823,0.041551,2019-05-05,396,189
33628,"id=99,timeshift=2019-05-12 00:00:00",1724976.0,0.0,0.0,8754.0,-0.011440,0.195194,1231.0,-0.010020,0.218883,20684465.0,0.307660,0.026701,716172336.0,0.593192,0.041506,2019-05-12,264,189


In [21]:
df= x.drop(['id'],axis=1)
import datetime as dt
df['Year'] = df.Date.dt.year
df["Quarter"] = df.Date.dt.quarter
df["Month"] = df.Date.dt.month

In [23]:
df.to_csv("C:/Users/rahul.kondhalkar/Downloads/Accelerators/Timeseries.csv")

In [24]:
# Simple Linear regression

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
linear_regressor=LinearRegression()
from sklearn import metrics

y=df.Target
X=df.drop(['Target','Date'],axis=1)



In [18]:
X_train,X_test, y_train, y_test = train_test_split(X,y)
linear_regressor.fit(X_train,y_train)
y_pred=linear_regressor.predict(X_test)
scores=[]

In [19]:
scores.append(metrics.explained_variance_score(y_test,y_pred))
scores.append(metrics.max_error(y_test,y_pred))
scores.append(metrics.mean_absolute_error(y_test,y_pred))
scores.append(metrics.mean_squared_error(y_test,y_pred))
scores.append(metrics.mean_squared_log_error(abs(y_test),abs(y_pred)))
scores.append(metrics.median_absolute_error(y_test,y_pred))
scores.append(metrics.r2_score(y_test,y_pred))
scores.append(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

accuracy_test=["explained_variance","max_error","neg_mean_absolute_error","neg_mean_squared_error","neg_mean_squared_log_error","neg_median_absolute_error","r2_score","rmse"]
score=dict(zip(accuracy_test,scores))

In [20]:
score

{'explained_variance': 0.029495738848782538,
 'max_error': 13422.297708383587,
 'neg_mean_absolute_error': 536.0541916813127,
 'neg_mean_squared_error': 1087003.3771112647,
 'neg_mean_squared_log_error': 20.289616478664087,
 'neg_median_absolute_error': 335.80598084130906,
 'r2_score': 0.029439597463801537,
 'rmse': 1042.5945410902864}

In [26]:
#catboost

from catboost import CatBoostRegressor
cat_regressor=CatBoostRegressor()
cat_regressor.fit(X_train,y_train)
y_pred=cat_regressor.predict(X_test)
scores=[]
scores.append(metrics.explained_variance_score(y_test,y_pred))
scores.append(metrics.max_error(y_test,y_pred))
scores.append(metrics.mean_absolute_error(y_test,y_pred))
scores.append(metrics.mean_squared_error(y_test,y_pred))
scores.append(metrics.mean_squared_log_error(abs(y_test),abs(y_pred)))
scores.append(metrics.median_absolute_error(y_test,y_pred))
scores.append(metrics.r2_score(y_test,y_pred))
scores.append(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
score=dict(zip(accuracy_test,scores))


Learning rate set to 0.069329
0:	learn: 1040.9986848	total: 117ms	remaining: 1m 57s
1:	learn: 1023.1343197	total: 138ms	remaining: 1m 8s
2:	learn: 1007.5298353	total: 169ms	remaining: 56s
3:	learn: 993.6783984	total: 189ms	remaining: 47s
4:	learn: 975.4748569	total: 229ms	remaining: 45.5s
5:	learn: 963.5071076	total: 244ms	remaining: 40.4s
6:	learn: 949.4971713	total: 263ms	remaining: 37.3s
7:	learn: 934.0113022	total: 278ms	remaining: 34.4s
8:	learn: 923.2368550	total: 297ms	remaining: 32.7s
9:	learn: 913.0183449	total: 314ms	remaining: 31.1s
10:	learn: 902.5470335	total: 325ms	remaining: 29.2s
11:	learn: 890.5685517	total: 341ms	remaining: 28.1s
12:	learn: 881.9262055	total: 366ms	remaining: 27.8s
13:	learn: 872.0808270	total: 377ms	remaining: 26.6s
14:	learn: 855.7256380	total: 392ms	remaining: 25.7s
15:	learn: 846.5734863	total: 404ms	remaining: 24.9s
16:	learn: 840.1325658	total: 416ms	remaining: 24s
17:	learn: 831.3380149	total: 433ms	remaining: 23.6s
18:	learn: 822.1608909	total

154:	learn: 475.5373449	total: 2.53s	remaining: 13.8s
155:	learn: 474.5977225	total: 2.54s	remaining: 13.8s
156:	learn: 474.1171074	total: 2.56s	remaining: 13.7s
157:	learn: 473.4701002	total: 2.57s	remaining: 13.7s
158:	learn: 473.2179847	total: 2.58s	remaining: 13.7s
159:	learn: 471.8451326	total: 2.59s	remaining: 13.6s
160:	learn: 471.0334776	total: 2.61s	remaining: 13.6s
161:	learn: 468.8091623	total: 2.62s	remaining: 13.6s
162:	learn: 468.2266284	total: 2.64s	remaining: 13.6s
163:	learn: 466.5747354	total: 2.65s	remaining: 13.5s
164:	learn: 465.8515159	total: 2.67s	remaining: 13.5s
165:	learn: 463.7605718	total: 2.68s	remaining: 13.5s
166:	learn: 463.2518962	total: 2.69s	remaining: 13.4s
167:	learn: 461.7828852	total: 2.71s	remaining: 13.4s
168:	learn: 461.1979943	total: 2.72s	remaining: 13.4s
169:	learn: 460.5985361	total: 2.75s	remaining: 13.4s
170:	learn: 459.0754598	total: 2.76s	remaining: 13.4s
171:	learn: 458.6102653	total: 2.77s	remaining: 13.3s
172:	learn: 458.0518698	tota

307:	learn: 383.5415412	total: 4.69s	remaining: 10.5s
308:	learn: 383.3681174	total: 4.7s	remaining: 10.5s
309:	learn: 383.1151917	total: 4.71s	remaining: 10.5s
310:	learn: 382.9683171	total: 4.72s	remaining: 10.5s
311:	learn: 382.4482808	total: 4.74s	remaining: 10.4s
312:	learn: 382.0736894	total: 4.75s	remaining: 10.4s
313:	learn: 381.4711818	total: 4.78s	remaining: 10.4s
314:	learn: 380.9034319	total: 4.8s	remaining: 10.4s
315:	learn: 380.3127725	total: 4.82s	remaining: 10.4s
316:	learn: 379.8871456	total: 4.83s	remaining: 10.4s
317:	learn: 379.4105336	total: 4.84s	remaining: 10.4s
318:	learn: 379.0458045	total: 4.85s	remaining: 10.4s
319:	learn: 378.5449170	total: 4.87s	remaining: 10.3s
320:	learn: 378.3417418	total: 4.9s	remaining: 10.4s
321:	learn: 378.1025014	total: 4.91s	remaining: 10.3s
322:	learn: 377.7064961	total: 4.92s	remaining: 10.3s
323:	learn: 377.3727758	total: 4.94s	remaining: 10.3s
324:	learn: 377.2430540	total: 4.96s	remaining: 10.3s
325:	learn: 377.0108380	total: 

460:	learn: 341.5948560	total: 7s	remaining: 8.18s
461:	learn: 341.4389782	total: 7.01s	remaining: 8.17s
462:	learn: 341.3020140	total: 7.03s	remaining: 8.15s
463:	learn: 341.2019026	total: 7.04s	remaining: 8.13s
464:	learn: 341.0846181	total: 7.06s	remaining: 8.13s
465:	learn: 340.6630712	total: 7.08s	remaining: 8.11s
466:	learn: 340.4547098	total: 7.1s	remaining: 8.11s
467:	learn: 340.2058759	total: 7.12s	remaining: 8.09s
468:	learn: 339.9246361	total: 7.14s	remaining: 8.09s
469:	learn: 339.7188637	total: 7.16s	remaining: 8.07s
470:	learn: 339.4429401	total: 7.17s	remaining: 8.05s
471:	learn: 339.3101735	total: 7.18s	remaining: 8.04s
472:	learn: 338.9992470	total: 7.2s	remaining: 8.02s
473:	learn: 338.7876100	total: 7.23s	remaining: 8.03s
474:	learn: 338.7056630	total: 7.25s	remaining: 8.01s
475:	learn: 338.5850149	total: 7.27s	remaining: 8s
476:	learn: 338.4184916	total: 7.29s	remaining: 7.99s
477:	learn: 338.2710716	total: 7.3s	remaining: 7.97s
478:	learn: 338.0479825	total: 7.32s	

613:	learn: 316.4751051	total: 9.37s	remaining: 5.89s
614:	learn: 316.3746731	total: 9.38s	remaining: 5.87s
615:	learn: 316.2128237	total: 9.39s	remaining: 5.86s
616:	learn: 316.1278184	total: 9.42s	remaining: 5.85s
617:	learn: 316.0305840	total: 9.43s	remaining: 5.83s
618:	learn: 315.8611813	total: 9.44s	remaining: 5.81s
619:	learn: 315.7711792	total: 9.46s	remaining: 5.79s
620:	learn: 315.4697313	total: 9.47s	remaining: 5.78s
621:	learn: 315.0648060	total: 9.48s	remaining: 5.76s
622:	learn: 314.9661883	total: 9.49s	remaining: 5.74s
623:	learn: 314.7998792	total: 9.5s	remaining: 5.73s
624:	learn: 314.7326108	total: 9.52s	remaining: 5.71s
625:	learn: 314.3997138	total: 9.53s	remaining: 5.69s
626:	learn: 314.2898762	total: 9.54s	remaining: 5.68s
627:	learn: 314.0878910	total: 9.55s	remaining: 5.66s
628:	learn: 313.8602809	total: 9.56s	remaining: 5.64s
629:	learn: 313.6556564	total: 9.58s	remaining: 5.63s
630:	learn: 313.5568423	total: 9.59s	remaining: 5.61s
631:	learn: 313.4561979	total

766:	learn: 297.1739982	total: 11.5s	remaining: 3.51s
767:	learn: 297.0473179	total: 11.6s	remaining: 3.49s
768:	learn: 296.9706081	total: 11.6s	remaining: 3.48s
769:	learn: 296.7267521	total: 11.6s	remaining: 3.46s
770:	learn: 296.5793503	total: 11.6s	remaining: 3.45s
771:	learn: 296.5076760	total: 11.6s	remaining: 3.44s
772:	learn: 296.3523101	total: 11.6s	remaining: 3.42s
773:	learn: 296.1967556	total: 11.7s	remaining: 3.4s
774:	learn: 296.1193332	total: 11.7s	remaining: 3.39s
775:	learn: 296.0389510	total: 11.7s	remaining: 3.37s
776:	learn: 295.9928156	total: 11.7s	remaining: 3.36s
777:	learn: 295.8425948	total: 11.7s	remaining: 3.34s
778:	learn: 295.7536380	total: 11.7s	remaining: 3.32s
779:	learn: 295.5030252	total: 11.7s	remaining: 3.31s
780:	learn: 295.4228621	total: 11.7s	remaining: 3.29s
781:	learn: 295.3241309	total: 11.8s	remaining: 3.28s
782:	learn: 295.2646684	total: 11.8s	remaining: 3.26s
783:	learn: 295.1254806	total: 11.8s	remaining: 3.25s
784:	learn: 295.0164016	total

919:	learn: 279.4552000	total: 13.9s	remaining: 1.21s
920:	learn: 279.3805833	total: 13.9s	remaining: 1.19s
921:	learn: 279.2074884	total: 13.9s	remaining: 1.18s
922:	learn: 279.1616928	total: 13.9s	remaining: 1.16s
923:	learn: 279.1170547	total: 13.9s	remaining: 1.14s
924:	learn: 278.7991169	total: 13.9s	remaining: 1.13s
925:	learn: 278.7030388	total: 13.9s	remaining: 1.11s
926:	learn: 278.6190219	total: 14s	remaining: 1.1s
927:	learn: 278.4824314	total: 14s	remaining: 1.08s
928:	learn: 278.4082715	total: 14s	remaining: 1.07s
929:	learn: 278.3435537	total: 14s	remaining: 1.05s
930:	learn: 278.2646277	total: 14s	remaining: 1.04s
931:	learn: 278.2141496	total: 14s	remaining: 1.02s
932:	learn: 278.1339467	total: 14.1s	remaining: 1.01s
933:	learn: 278.0576191	total: 14.1s	remaining: 995ms
934:	learn: 277.9556495	total: 14.1s	remaining: 979ms
935:	learn: 277.9027382	total: 14.1s	remaining: 965ms
936:	learn: 277.8044453	total: 14.1s	remaining: 950ms
937:	learn: 277.6433060	total: 14.1s	rema

{'explained_variance': 0.9055154816878602,
 'max_error': 8648.038799651698,
 'neg_mean_absolute_error': 150.44388629557955,
 'neg_mean_squared_error': 105835.10847633945,
 'neg_mean_squared_log_error': 9.007942131621789,
 'neg_median_absolute_error': 66.44199915411107,
 'r2_score': 0.9055022572623121,
 'rmse': 325.32308322087977}

In [27]:
print("Catboost Validation results")
score

Catboost


{'explained_variance': 0.9055154816878602,
 'max_error': 8648.038799651698,
 'neg_mean_absolute_error': 150.44388629557955,
 'neg_mean_squared_error': 105835.10847633945,
 'neg_mean_squared_log_error': 9.007942131621789,
 'neg_median_absolute_error': 66.44199915411107,
 'r2_score': 0.9055022572623121,
 'rmse': 325.32308322087977}

In [ ]:
# To imrove more score we can more data preprocessing, Feature engineering and Hyperparameter Optimization also we can try different algorithm
# Also for better model explanation we can add LIME and SHAPLEY